In [1]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("nielsr/funsd")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/755 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/4.38M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/149 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50 [00:00<?, ? examples/s]

In [2]:
pip install transformers datasets torch pillow pytesseract


In [3]:
from datasets import load_dataset

dataset = load_dataset("nielsr/funsd")

train_dataset = dataset["train"]
test_dataset = dataset["test"]

print(train_dataset[0])


{'id': '0', 'words': ['R&D', ':', 'Suggestion:', 'Date:', 'Licensee', 'Yes', 'No', '597005708', 'R&D', 'QUALITY', 'IMPROVEMENT', 'SUGGESTION/', 'SOLUTION', 'FORM', 'Name', '/', 'Phone', 'Ext.', ':', 'M.', 'Hamann', 'P.', 'Harper,', 'P.', 'Martinez', '9/', '3/', '92', 'R&D', 'Group:', 'J.', 'S.', 'Wigand', 'Supervisor', '/', 'Manager', 'Discontinue', 'coal', 'retention', 'analyses', 'on', 'licensee', 'submitted', 'product', 'samples', '(Note', ':', 'Coal', 'Retention', 'testing', 'is', 'not', 'performed', 'by', 'most', 'licensees.', 'Other', 'B&W', 'physical', 'measurements', 'as', 'ends', 'stability', 'and', 'inspection', 'for', 'soft', 'spots', 'in', 'ciparettes', 'are', 'thought', 'to', 'be', 'sufficient', 'measures', 'to', 'assure', 'cigarette', 'physical', 'integrity.', 'The', 'proposed', 'action', 'will', 'increase', 'laboratory', 'productivity', '.', ')', 'Suggested', 'Solutions', '(s)', ':', 'Delete', 'coal', 'retention', 'from', 'the', 'list', 'of', 'standard', 'analyses', 'per

In [4]:
LABELS = [
    "O",
    "B-QUESTION", "I-QUESTION",
    "B-ANSWER", "I-ANSWER",
    "B-HEADER", "I-HEADER"
]

label2id = {label: i for i, label in enumerate(LABELS)}
id2label = {i: label for label, i in label2id.items()}


In [9]:
from transformers import LayoutLMv3Processor
import torch # Import torch for .squeeze()

processor = LayoutLMv3Processor.from_pretrained(
    "microsoft/layoutlmv3-base",
    apply_ocr=False
)

def preprocess(example):
    image = example["image"].convert("RGB") # Convert image to RGB
    encoding = processor(
        image,
        example["words"],
        boxes=example["bboxes"],
        word_labels=example["ner_tags"],
        truncation=True,
        padding="max_length",
        return_tensors="pt" # Ensure tensors are returned
    )

    # The processor returns tensors with a batch dimension of 1 for a single input.
    # We need to remove this for each individual example before batching by Trainer's collator.
    for k, v in encoding.items():
        if isinstance(v, torch.Tensor) and v.ndim > 1 and v.shape[0] == 1:
            encoding[k] = v.squeeze(0)

    return encoding

encoded_train = train_dataset.map(
    preprocess,
    batched=False,
    remove_columns=train_dataset.column_names
)

encoded_test = test_dataset.map(
    preprocess,
    batched=False,
    remove_columns=test_dataset.column_names
)

Map:   0%|          | 0/149 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [12]:
from transformers import LayoutLMv3ForTokenClassification
from transformers import Trainer, TrainingArguments, DefaultDataCollator

model = LayoutLMv3ForTokenClassification.from_pretrained(
    "microsoft/layoutlmv3-base",
    num_labels=len(LABELS),
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="./funsd_model",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    eval_strategy="steps",
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    report_to="none"
)

data_collator = DefaultDataCollator()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_test,
    data_collator=data_collator # Use DefaultDataCollator for preprocessed data
)

trainer.train()

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:1621: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Step,Training Loss,Validation Loss
100,1.112700,0.671565
200,0.661500,0.585828


TrainOutput(global_step=225, training_loss=0.8458343972100152, metrics={'train_runtime': 5841.0771, 'train_samples_per_second': 0.077, 'train_steps_per_second': 0.039, 'total_flos': 117831588424704.0, 'train_loss': 0.8458343972100152, 'epoch': 3.0})

In [17]:
import torch

def infer(example):
    model.eval()

    # Convert the image to RGB before processing, similar to the preprocess function
    image_rgb = example["image"].convert("RGB")

    encoding = processor(
        image_rgb,
        example["words"],
        boxes=example["bboxes"],
        return_tensors="pt",
        truncation=True,
        padding="max_length"
    )

    with torch.no_grad():
        outputs = model(**encoding)

    predictions = outputs.logits.argmax(-1).squeeze().tolist()

    results = []
    for word, label_id in zip(example["words"], predictions):
        results.append({
            "word": word,
            "label": id2label[label_id]
        })

    return results

In [20]:

import torch
from PIL import Image
import numpy as np

def infer(example):
    model.eval()

    image = example["image"]

    # FIX: ensure image is RGB (3 channels)
    if isinstance(image, Image.Image):
        image = image.convert("RGB")
    elif isinstance(image, np.ndarray):
        if image.ndim == 2:  # grayscale
            image = np.stack([image]*3, axis=-1)
        image = Image.fromarray(image).convert("RGB")

    encoding = processor(
        image,
        example["words"],
        boxes=example["bboxes"],
        return_tensors="pt",
        truncation=True,
        padding="max_length"
    )

    with torch.no_grad():
        outputs = model(**encoding)

    predictions = outputs.logits.argmax(-1).squeeze().tolist()

    results = []
    for word, label_id in zip(example["words"], predictions):
        results.append((word, id2label[label_id]))

    return results


# RUN THIS
sample = test_dataset[0]
predictions = infer(sample)

for w, l in predictions:
    print(w, "->", l)


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:1621: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


TO: -> O
DATE: -> B-ANSWER
3 -> B-ANSWER
Fax: -> B-ANSWER
NOTE: -> B-ANSWER
82092117 -> I-ANSWER
614 -> B-HEADER
-466 -> O
-5087 -> O
Dec -> O
10 -> B-ANSWER
'98 -> I-ANSWER
17 -> O
:46 -> O
P. -> O
01 -> O
ATT. -> O
GEN. -> O
ADMIN. -> I-HEADER
OFFICE -> O
Attorney -> I-HEADER
General -> I-HEADER
Betty -> O
D. -> O
Montgomery -> O
CONFIDENTIAL -> O
FACSIMILE -> O
TRANSMISSION -> O
COVER -> O
SHEET -> O
(614) -> O
466- -> O
5087 -> O
FAX -> O
NO. -> O
George -> I-QUESTION
Baroody -> I-QUESTION
(336) -> I-QUESTION
335- -> I-QUESTION
7392 -> I-QUESTION
FAX -> I-QUESTION
NUMBER: -> B-HEADER
PHONE -> I-HEADER
NUMBER: -> B-HEADER
(336) -> I-HEADER
335- -> I-HEADER
7363 -> I-HEADER
NUMBER -> B-QUESTION
OF -> I-QUESTION
PAGES -> I-QUESTION
INCLUDING -> I-QUESTION
COVER -> I-QUESTION
SHEET: -> I-QUESTION
June -> I-QUESTION
Flynn -> I-QUESTION
for -> I-QUESTION
Eric -> I-QUESTION
Brown/ -> I-QUESTION
(614) -> I-QUESTION
466- -> I-QUESTION
8980 -> O
SENDER -> B-HEADER
/PHONE -> O
NUMBER: -> I-HE